In [1]:
import numpy as np
file = "CMU_clips_no_zip.npy"
X = np.load(file, allow_pickle=True)
X = np.reshape(X, (X.shape[0], X.shape[1],X.shape[2]*X.shape[3]))
X = X/np.max(X)

(1073, 3198, 31, 6)


In [9]:
from sklearn.model_selection import train_test_split
import random
import numpy as np
randomlist = []
for i in range(0,1073):
    n = float(random.randint(0,1))
    randomlist.append(n)
list = np.array(randomlist)
del randomlist
X_train,X_test,y_train,y_test = train_test_split(X,list,test_size = 0.2, random_state = 42)

[1. 1. 1. ... 1. 0. 1.]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import CuDNNLSTM,Dense,Dropout
from tensorflow.keras import utils

nNeurons = 186
output_layers = 1
# creating the NN model
model = Sequential()
model.add( CuDNNLSTM( nNeurons, return_sequences = True, input_shape = (3198, 186) ) )
model.add( Dropout(0.2))
model.add( CuDNNLSTM( nNeurons ) )
model.add( Dropout(0.2))
model.add( Dense( output_layers, activation = 'softmax' ) )

opt = tf.keras.optimizers.Adam(lr =1e-3, decay=1e-5)
model.compile( loss = 'sparse_categorical_crossentropy',  optimizer = opt, metrics = ['accuracy'] )
model.summary()

probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-biggeer.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode = 'min')
callbacks_list = [checkpoint]

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
model.fit(X_train, y_train, epochs = 30,validation_data=(X_test,y_test), batch_size=32, callbacks=callbacks_list)
#model.save('cmuclassifier.model')

y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))


a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=100, batch_size=64)


plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()